# A Transition State

Rotation around a dihedral is a lot like a chemical reaction. We started in one energy well, passed through a high-energy barrier, and ended up in a different energy well. In fact, its exactly like a chemical reaction. We can optimize for the structures in the energy wells as the algorithm just has to follow the derivative of the energy function downhill. To optimize for the structure that represents the high energy transition state between the two energy wells we will need to perform a transtition state optimization.

## Using the Imagination

To optimize for a transition state (ts) we must first start very close to it. There's all kinds of ways to go uphill in energy. Just grab any bond and pull. However, we want to follow the path that leads to the ts. First we start with a structure that is close enough to the ts to have an imaginary vibration. We perform a vibrational analysis on the structure, identify the imaginary vibration, and then follow it to the crest of the energy barrier. Yes its hard math, that is why we are letting a computer do it for us.

## A Starting Point

We will use the rotation of butane as our example. We have already performed a PES and observed an energy maximum near (but not exactly at) 120 degrees. Let us start there. First we will optimize the structure of butane with a torsion angle of 120 degrees. consider the code below. 

In [1]:
# use psi4conda environment
import psi4
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import helpers as hp # Many useful functions from https://lcbc-epfl.github.io/iesm-public/intro.html

psi4.core.clean_options()

output_file = "butane_TS_Start_120.log"

psi4.set_memory("2GB")
psi4.set_output_file(output_file, append=False, loglevel=20, print_header=True, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)

# The Z-matrix as a text string ### From previous optimization
data = """
       0 1
       C       
       C             1    1.541090
       H             1    1.085997      2  110.737654
       H             1    1.086006      2  110.607207      3  119.996707
       H             1    1.085996      2  110.737537      3 -120.006541
       C             2    1.545066      1  112.446725      3   59.916254
       H             2    1.088519      1  109.298807      6  121.516910
       H             2    1.088518      1  109.301738      6 -121.519264
       C             6    1.541169      2  112.445221      1    dihedral
       H             6    1.088515      2  109.250918      9  121.551235
       H             6    1.088516      2  109.250874      9 -121.547269
       H             9    1.085998      6  110.604358     10   58.384210
       H             9    1.085992      6  110.745577     12  119.994957
       H             9    1.085994      6  110.746117     12 -119.995065

       symmetry c1
       dihedral  =  120
       units angstrom
       """ 

# Create the Molecule object
mol = psi4.geometry(data)             # Create Molecule object from data string

# Set up options for calculations
psi4.set_options({
#       "BASIS": "sto-3g",            # default => None - Basis set must be specified
#       "BASIS": "6-31+G(d)",         # I am sepcifying the basis set in the method string of the optimize function
#       "BASIS": "6-31++G(d,p)",      #  so I do not need to set the "BASIS" option here.
#        "SAVE_OPTIMIZATION": True,    # default => False
#        "OPT_TYPE": "min",            # default => "min":  MIN, TS, IRC
#        "MAXITER": 100,               # default => 50
#        "GEOM_MAXITER": 100,          # default => 50
#        "FULL_HESS_EVERY": -1,        # default => -1 -> No Hessian performed. 0 => perform a calculation at the beginning
#        "PRINT": 2,                   # default => 1
#        "GUESS": "sad",               # default => "auto": AUTO, CORE, GWH, SAD, SADNO, SAP, SAPGAU, HUCKEL, MODHUCKEL, READ
#        "REFERENCE": "rhf",           # default => rhf: RHF, ROHF, UHF, CUHF, RKS, UKS
#        "SCF_TYPE": "direct",         # default => pk:  DIRECT, DF, PK, OUT_OF_CORE, PS, INDEPENDENT, GTFOCK, DFDIRJ+LINK, DFDIRJ+COSX
#        "INTS_TOLERANCE": 1E-8,       # default => 1e-12. A value of 1e-8 is recommended when SCF_TYPE set to "direct"
#        "PRINT_TRAJECTORY_XYZ_FILE":True,   # default => false
#        "PRINT_OPT_PARAMS": True,     # default => False
#        "WRITE_TRAJECTORY": True,     # default => False
    })


# Optimize the structure
psi4.set_options({"FROZEN_DIHEDRAL":"9 6 2 1"})

energy_ts, hist = psi4.optimize("hf/sto-3g", molecule = mol, return_history=True)
                                                                        
print(f"Initial energy is {energy_ts:0.7f} Hartrees")

dihedral_angle = mol.get_variable("dihedral")

print(f"After optimization: {dihedral_angle:.3f} degrees\n") 

hp.drawXYZ(mol)


  Memory set to   1.863 GiB by Python driver.
Optimizer: Optimization complete!
Initial energy is -155.4617513 Hartrees
After optimization: 120.000 degrees



3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## The Transition State for Rotation

We will now perform a transition state optimization. Note the ```"OPT_TYPE": "ts"``` setting in the code below. The optimized dihedral angle was 119.85 degrees using hf/STO-3G (10 seconds to run). This was surprising to me. I expected it to be slightly higher than 120 degrees due to steric repulsion. Perhaps hf/STO-3G is not accurate enough. 

The hf/STO-3G transition state optimization took abot 10 seconds on my laptop computer. I repeated the caluclation using hf/6-31+G(d) and the angle was optimized to be 119.0 degrees. The higher level of theory favoured even more cheating toward the other methyl group. This level of theory took a full minute to optimize.

### Further Notes:

Note all the comment-out options in the code below. Use the appropriate oprions and methods to repeat the calculations described here. 

I used a higher level of theory again, b3lyp/6-31+G(d). This is a density functional theory (DFT) method. DfT is better are handling charged systems and weak interactions (I think, I am not a computational chemist. This higher level of theory took 14 minutes to optimizer the transition state and found a dihedral angle of 188.5 degrees.

However b3lyp does not include dispersion interaction in it math. This becomes important in interactions between molecules and in groups between molecules. I performed a B3LYP-d3bj2b/6-311++G(d,p) transition state oprimization (22 minutes) and the angle was found to be 119.5 degrees.

The steric interaction appears to be unimportant between the methyl groups in the ts. The reason that the transition state is reached slightly before 120 degrees is explained using Hammond's postulate. Look at the energy profile in the previous notebook and convince yourself that this observation is consistent with Hammond's postulate.

In [2]:
ts = mol.clone()
psi4.core.clean_options()

output_file = "butane_TS.log"
psi4.set_memory("2GB")
psi4.set_output_file(output_file, append=False, loglevel=20, print_header=True, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)

psi4.set_options({
#        "BASIS": "sto-3g",            # default => None - Basis set must be specified
#        "BASIS": "6-31+G(d)",         # I am sepcifying the basis set in the method string of the optimize function
#        "BASIS": "6-31++G(d,p)",      #  so I do not need to set the "BASIS" option here.
#        "BASIS": "6-311++G(d,p)",        
#        "BASIS": "cc-pvdz"
#        "SAVE_OPTIMIZATION": True,    # default => False
        "OPT_TYPE": "ts",              # default => "min":  MIN, TS, IRC
#        "MAXITER": 100,               # default => 50
#        "GEOM_MAXITER": 500,          # default => 50
        "FULL_HESS_EVERY": 0,         # default => -1 -> Does not do Hessian; 0 -> hessian at start
#        "PRINT": 2,                   # default => 1
#        "GUESS": "sad",               # default => "auto": AUTO, CORE, GWH, SAD, SADNO, SAP, SAPGAU, HUCKEL, MODHUCKEL, READ
#        "REFERENCE": "rhf",            # default => rhf: RHF, ROHF, UHF, CUHF, RKS, UKS
#        "SCF_TYPE": "direct",         # default => pk:  DIRECT, DF, PK, OUT_OF_CORE, PS, INDEPENDENT, GTFOCK, DFDIRJ+LINK, DFDIRJ+COSX
#        "SCF_TYPE": "df",             # default => pk
#        "INTS_TOLERANCE": 1E-8,       # default => 1e-12. A value of 1e-8 is recommended when SCF_TYPE set to "direct"
#        "PRINT_TRAJECTORY_XYZ_FILE":True,   # default => false
#        "PRINT_OPT_PARAMS": True,      # default => False
#        "WRITE_TRAJECTORY": True,      # default => False
        "NORMAL_MODES_WRITE": True,    # default => False
    })

energy_ts, hist = psi4.optimize("hf/sto-3g", molecule = ts, return_history=True)          
#energy_ts, hist = psi4.optimize("b3pw91/6-31+G(d)", molecule = ts, return_history=True)   
#energy_ts, hist = psi4.optimize("b3lyp/6-311++G(d,p)", molecule = ts, return_history=True)   
#energy_ts, hist = psi4.optimize("B3LYP-d3bj2b/6-311++G(d,p)", molecule = ts, return_history=True)   

#print out TS in case something goes wrong
with open('ts.xyz', 'w') as f:
    f.write(ts.save_string_xyz_file())

print(f"TS energy is {energy_ts:0.7f} Hartrees")

dihedral_angle = ts.get_variable("dihedral")
print(f"After TS optimization dihedral is: {dihedral_angle:.3f} degrees\n") 
print("done")
hp.drawXYZ(ts)

Optimizer: Optimization complete!
TS energy is -155.4617512 Hartrees
After TS optimization dihedral is: 119.826 degrees

done


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### What's the Frequency, Kenneth?

A transition state will have an imaginary frequency that represents the motion across the energy surface from starting condition to ending position. Can we observe such an imaginary frequency here?

We will perform a frequency calculation and add the option to write out a file that contains the frequency "modes" in a format that we can use for visualization. In the code above we had set the option ```show_normal_modes``` to ```True```. This performs a frequency calculation and writes a Molden-formatted frequency file. You will find a file named ```some_filename.molden_normal_modes``` in your working directory.  

We will use the ```helpers``` package written by Ursula Roethlisberger and Andrej Antalík of the École Polytechnique Fédérale de Lausanne to visualize the frequencies.

Consider the code below. We have found the vibration data file in the directory and copied its filename into the ```show_normal_modes``` function from the ```helpers``` package. We observe that the imaginary frequency (the negative frequency) indeed describes rotation around the dihedral angle defined for bond rotation. Try the other frequencies. Do they track with what you learned in spectroscopy?

We conclude that the transition state structure is likely correct (we're not saying its accurate, just that it is consistent with our hypothesis.)

In [12]:
!ls -alr *.molden_normal_modes

-rw-r--r--  1 blink  staff  34528 15 May 09:49 butane_TS.default.29456.molden_normal_modes
-rw-r--r--  1 blink  staff  34528 15 May 09:48 butane_TS.default.1122.molden_normal_modes


In [10]:
# Find the file written for the calculation above with 
# the extension ".molden_normal_modes" and enter
# the file name in the command below.

hp.show_normal_modes(filename='butane_TS.default.29456.molden_normal_modes')

interactive(children=(Dropdown(description='Normal mode:', options=((-129.1479563951, 0), (249.2734416537, 1),…